In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

from sklearn.model_selection import train_test_split

### Data extraction
extract current data from the database and prepare for processing

In [3]:
# TODO: extract data and target

X = []  # TODO: choose scaler
Y = []

### Data preparation
Split between training and testing set

In [4]:
from torch.utils.data import TensorDataset, DataLoader

train_x, test_x, train_y, test_y = train_test_split(X, Y, test_size=0.2, random_state=1)

test_x = torch.Tensor(test_x)
test_y = test_y  # TODO: choose encoding (one_hot, etc.)

train_dataset = TensorDataset(train_x, train_y)
test_dataset = TensorDataset(test_x, test_y)
train_loader = DataLoader(train_dataset)
test_loader = DataLoader(test_dataset)

classes = ['asphalt', 'gravel', 'nature']  # TODO: define all classes

ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

### Building the Model

In [ ]:
class SurfaceModel(nn.Module):
    def __init__(self):
        super(SurfaceModel, self).__init__()
        # Todo: Define

    def forward(self, x):
        # Todo: Apply
        return x


model = SurfaceModel()

### Training the Model

In [ ]:
lr = 1e-2
momentum = 0.9
num_epochs = 100

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")  # for testing on my Mac

criterion = nn.CrossEntropyLoss()
# criterion = nn.MSELoss()
# criterion = nn.NLLLoss()

optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
# optimizer = optim.Adam(model.parameters(), lr=lr)

print("Training STARTED")
model.to(device)
for e in range(0, num_epochs):
    model.train()  # set the model in training mode
    totalTrainLoss = 0  # initialize the total training and validation loss
    totalValLoss = 0
    # initialize the number of correct predictions in the training and validation step
    trainCorrect = 0
    valCorrect = 0

    for i, (x, y) in enumerate(train_loader):  # loop over the training set
        (x, y) = (x.to(device), y.to(torch.float32).to(device))  # send the input to the device
        pred = model(x)  # perform a forward pass and calculate the training loss

        loss = criterion(pred, y)
        # loss = criterion(m(pred), y) # for NLLLoss()
        
        # zero out the gradients, perform the backpropagation step, and update the weights
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # add the loss to the total training loss so far and calculate the number of correct predictions
        totalTrainLoss += loss

    if (e + 1) % 10 == 0 | e == 0:
        print("Epoch", e, "Training Loss:", totalTrainLoss.item())

print("Training FINISHED")

## Testing

In [ ]:
import numpy as np
import util

### Training Results

In [ ]:
training_loss, class_correct, class_total = util.compute_accuracy(model, train_loader, device, criterion)

# average training loss
training_loss = training_loss/len(train_loader.dataset)
print('Training Loss: {:.6f}\n'.format(training_loss))

for i in range(10):
    if class_total[i] > 0:
        print('Training Accuracy of %5s: %2d%% (%2d/%2d)' % (
            classes[i], 100.0 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Training Accuracy of %5s: N/A ' % (classes[i]))

print('\Training Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))

### Testing Results

In [ ]:
test_loss, class_correct, class_total = util.compute_accuracy(model, test_loader, device, criterion)

# average test loss
test_loss = test_loss/len(test_loader.dataset)
print('Test Loss: {:.6f}\n'.format(test_loss))

for i in range(10):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            classes[i], 100.0 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))